# Project Wrangling

## Data Gathering

<div align="justify">There are three data sources that I used for this project. The first is the WeRateDogs Twitter archive data which I already downloaded in a csv format. The second is the tweet image prediction that I downloaded using the  `request` package in python and write it to a tsv file. For the last part I used the twitter api to download the tweet details of all the tweets from the twitter archive data that I initially downloaded. First to protect my API tokens I used `configparser` package in python to get my API tokens from a configfile. Then I initialize the API, get the tweet_ids from the twitter_archive file, create a function that would use the tweeter APU to get the details corresponding to each tweet_ids and store it to a .json file and then run the function. Of the 2356 tweet_id, there was an error in the retrieval for 32 of them. Finally, I created a pandas dataframe for the three file.</div>

## Assessing Data

<div align="justify"><p>I first assessed the data visually by looking the three dataframes and try to look for things that I would focus on when I proceed to the programatic assessment. On my visual assessment, I noticed that there are columns that have a lot of null values, for dog name I noticed thare there are entries that are not a name such as 'a', there are duplicated columns and columns which entries are in the form of html code, lists and dictionary.</P>

<p>I then did the progrmatic assessment. I first use the function `.info()` to check for the number of non-null entries and the datatype for each of the columns of the three tables. After which I look at each of the columns. I check if there are duplicates for tweet_id, use the `.describe()` function on the numerical data and look at the unique items and distribution for cathegorical data. I also look at the sample entries for the columns that have lists or dictionaries as entries.</p></div>

The following are the lists of issues that I had identified.
* Quality Issues
1. The current tables still have retweets. There are 159 retweets from the tweet_from_api table and 181 tweets from twitter_archive.
2. The columns p1, p2, p3, p1_dog, p2_dog, p3_dog, p1_conf, p2_cof, p3_conf though giving a lot of information regarding the models used and the predictions, it is hard to see what dog breed the dog is most likely to be since they also include non-dog predictions and predictions that the model deemed false. As an alternative we could create 3 columns, the most likely dog_breed, the model used to determine the breed and the confidence interval for the prediction.
3. There are unneeded columns:
- There are columns that are empty or almost nearly empty: 
    - for empty columns: 
        - tweet_from_api_copy ['geo', 'coordinates', 'contributors'] 
    - for nearly empty:
        - tweet_from_api_copy['place'] has only 1 non null entry
- There are columns that are duplicates:
    - The following columns are duplicates: 
        - 'id_str', 'extended_enteties', 'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'possibly_sensitive_appealabl
4. The following columns should be a string ['tweet_id','in_reply_to_status_id','in_reply_to_user_id']
5. The column possibly_sensitive should be a bool.
6. The following items in the name of dogs 'a', 'actually', 'all', 'an', 'by', 'getting', 'his', 'incredibly', 'infuriating', 'just', 'life', 'light', 'mad', 'my', 'not', 'officially', 'old', 'one', 'quite', 'space', 'such', 'the', 'this', 'unacceptable', 'very' are not names.
7. The column source is in html format. There is a need to extract the source:
    - Twitter for iPhone
    - Twitter for Web Client
    - TweetDeck
8. The following columns have more than one value
    - The column display_text_range is a list. 
    - The column entities is a dictionary
    - The column user is a dictionary
* Tidiness issues
1. There are currently three tables but they could be joined into one. twitter_archive columns ['rating_numerator', 'rating_denominator', 'name', 'doggo', 'floofer','pupper', 'puppo', 'expanded_urls'], image_prediction columns could be joined with tweet_from_api
2. The kind of dog (doggo, floofer, pupper and puppo) should only be in one column

## Cleaning Data

<div align="justify"><p>In the start of my cleaning data, I created a copy of all three dataframes. At the end, I wanted to have a cleaned data on the original tweets with image. With this I first address the differences in the number of tweets among the three tables. I used the unique entries in tweet_id in the image_prediction to filter all three tables to come up with tables of only those with image. Then to filter out the retweets, I used the unique tweet_ids of the tweets that are null in the retweet_status_id and retweet_status columns as these two would indicate if the tweet is a retweet or not.</p>

<p>I then used the `.dropna()` function to remove to columns that are empty and then I proceeded to join the three tables together using the `.merge()` function on the `tweet_id` column. I ensured only to include the columns that is unique for each table. I removed the columns that were duplicates with the `.drop()` function and did the necessary change of the datatypes of columns using `.astype()` function.</p>

<p>The columns `doggo`, `floofer`, `pupper` and `puppo`, all pertains to the dog category and they could be merged into one. First I replace the 'None' entries to null values then joined the strings of each column into one. Then for those that resulted to an empty string, I replaced it with 'uncategorized'.</p>

<p>The columns `p1`, `p2`, `p3`, `p1_dog`, `p2_dog`, `p3_dog`, `p1_conf`, `p2_conf` and `p3_conf` are columns that describes how the breed of dogs was determined but the columns does not readily show what dog_breed the twitter image shows since there are predictions that the model deemed false. To come up with the predicted dog breed, first I created a function that would check from the first model to the third had a prediction that is `True` then from this input I would get the predicted dog breed from that model and its coefficient. In order not to loose the data from the original columns, I created another column from them.</p>

<p>What I find tricky is for the columns that have entries in the form of lists and dictionary. For these columns, I used the function `.to_list()` to create columns for each item in the list and the `pd.json_normalize()' that creates a column for each `key` in the dictionary. After expanding the columns, I retained in my working table only the most relevant columns that is directly related to the tweets other details such as user details, I stored in other tables.</p></div>